In [1]:
# %pip install beautifulsoup4
# %pip install scrapy
# %pip install requests

In [2]:
import scrapy
from urllib.parse import unquote
import json
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv

class LinkedJobsSpider(scrapy.Spider):
    name = "linkedin_jobs"
    api_url_template = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={}&location=United%2BStates&geoId=103644278&trk=public_jobs_jobs-search-bar_search-submit&start={}'
    jobs_url = []
    urls = []

    # custom_settings = {
	# 	'FEEDS': { 'url.csv': { 'format': 'csv',}}
	# 	}

    def start_requests(self):

        # keywords = ['software%20engineer', 'data%20scientist', 'data%20engineer', 'machine%20learning%20engineer', 'artificial%20intelligence%20engineer', 'python']
        keywords = 'python'#'software%2Bengineer'

        output_dir = 'data_url' 
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        search_keywords = unquote(keywords).replace('%2B', '_')
        filename = os.path.join(output_dir, '{}.csv'.format(search_keywords))  # Assuming 'title' exists in the item
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['url', 'job_code', 'job_id'])

        for page in range(3):
            start_value = page * 25
            url = self.api_url_template.format(keywords, start_value)
            # search_keywords = unquote(keywords).replace('%2B', '_')
            yield scrapy.Request(url=url, callback=self.parse_job, meta={'start_value': start_value, 'search_keywords': search_keywords})


    def parse_job(self, response):
        start_value = response.meta.get('start_value')
        search_keywords = response.meta.get('search_keywords')
  

        jobs = response.css("li")

        num_jobs_returned = len(jobs)
        # count += num_jobs_returned
        print(f"******* Num Jobs Returned for start={start_value} *******")
        print(num_jobs_returned)
        # print("total: {}".format(self.count))
        print('*****')

        output_dir = 'data_url' 
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        filename = os.path.join(output_dir, '{}.csv'.format(search_keywords))
        # filename = response.meta.get('filename')
        urls = []

        for job in jobs:
            # url_item = {}

            detail_url = job.css(".base-card__full-link::attr(href)").get(default='not-found').strip()

            url_parts = detail_url.split('/')
            jobs_index = url_parts.index('view')
            
            job_code = url_parts[jobs_index + 1].split("?refId")[0]
            job_id = job_code.split('-')[-1]

            url_item = {
                'url': detail_url,
                'job_code': job_code,
                'job_id': job_id
            }

            urls.append(url_item)

        df = pd.DataFrame(urls)
        with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
            df.to_csv(csvfile, header=not csvfile.tell(), index=False)
            # csv_writer = csv.writer(csvfile)
            
            # csv_writer.writerows(df)
                
        # print(urls)            
        # search_keywords = unquote(keywords).replace('%2B', '_')
        # filename = os.path.join(output_dir, '{}.csv'.format(search_keywords))  # Assuming 'title' exists in the item
        # with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        #     csv_writer = csv.writer(csvfile)
        #     csv_writer.writerow(['url', 'job_code', 'job_id'])

            
                
                # Check if file is empty, if yes, write header row
                
            






Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/dz/_kg4kpk160d653__f0w9lxbm0000gn/T/ipykernel_27016/2108451895.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
settings = get_project_settings()
process = CrawlerProcess(settings)
process.crawl(LinkedJobsSpider)
process.start()

2024-02-14 23:18:57 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2024-02-14 23:18:57 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.12.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.9.18 (main, Sep 11 2023, 08:38:23) - [Clang 14.0.6 ], pyOpenSSL 23.2.0 (OpenSSL 3.0.12 24 Oct 2023), cryptography 41.0.7, Platform macOS-10.16-x86_64-i386-64bit
2024-02-14 23:18:57 [scrapy.addons] INFO: Enabled addons:
[]
2024-02-14 23:18:57 [py.warnings] WARNING: /Users/yidan/opt/anaconda3/envs/deeplearn/lib/python3.9/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation

******* Num Jobs Returned for start=50 *******
24
*****
******* Num Jobs Returned for start=0 *******
24
*****
******* Num Jobs Returned for start=25 *******
25
*****
